In [1]:
import boto3
import pprint
from IPython.core.display import display, HTML

import email
import smtplib

import json
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
#Could Whitelist some port rules which are needed for some internal use and are a known risk
white_list = [{'rule_name': 'jupyter-datadog', 'port': 8889}, 
              {'rule_name': 'launch-wizard-17', 'port': 25},
              {'rule_name': 'launch-wizard-17', 'port': 465}]

In [3]:
email_smtp = 'smtp.gmail.com'
email_port = 587
email_id = 'hydranoob000@gmail.com'
email_password = 'hqkcvqarjdhmiqsq'
regions = ['ap-south-1']

In [4]:
def ec2_secg_check():
    x = ''
    for region in regions:
        client = boto3.client('ec2', region_name=region)
        response=client.describe_security_groups()
        for secg in response['SecurityGroups']:
            for ipr in secg['IpPermissions']:
                for cidr in ipr['IpRanges']:
                    if '0.0.0.0' in cidr['CidrIp']:
                        whitelist_status = ""
                        for i in white_list:
                            if i['rule_name'] == secg['GroupName'] and i['port'] == ipr['FromPort']:
                                whitelist_status = "Whitelisted"
                            else:
                                pass
                        if whitelist_status != "Whitelisted":
                            x = x + "Found malicious rule for port: "+str(ipr['FromPort']) + '<br>'
                            x = x + "Rule Name: "+secg['GroupName'] + "<br>"
                            x = x + "Link to rule: "+ "https://"+region+".console.aws.amazon.com/ec2/home?region="+region+"#SecurityGroups:search="+secg['GroupName']
    return x


In [5]:
msg = ec2_secg_check()

In [6]:
def send_email(sender_email, sender_name, email_subject, email_message,
                        recipient_list, cc_list=None, **kwargs):
    try:
        if cc_list is None:
            cc_list = ''

        cc_list = cc_list.split(',')
        recipient_list = recipient_list.split(',')

        msg = MIMEMultipart('alternative')
        msg['Subject'] = email_subject
        msg['From'] = email.utils.formataddr((sender_name, sender_email))
        msg['To'] = ','.join(recipient_list)

        if cc_list:
            msg['CC'] = ','.join(cc_list)
            recipient_list.extend(cc_list)


        part1 = MIMEText(email_message, 'plain')
        part2 = MIMEText(email_message, 'html')

        msg.attach(part1)
        msg.attach(part2)

        try:
            server = smtplib.SMTP(email_smtp, email_port)
            server.ehlo()
            server.starttls()
            server.ehlo()
            server.login(email_id, email_password)
            server.sendmail(sender_email, recipient_list, msg.as_string())
            return {'result': "OK!, Email has been sent to user successfully", 'execution_status': 'SUCCESS'}
        except Exception as e:
            return {'result': str(e), 'execution_status': 'ERROR'}
    except Exception as f:
        return {'result': str(f), 'execution_status': 'ERROR'}

In [7]:
msg

'Found malicious rule for port: 3389<br>Rule Name: hydra<br>Link to rule: https://ap-south-1.console.aws.amazon.com/ec2/home?region=ap-south-1#SecurityGroups:search=hydra'

In [8]:
# GET /ec2-sec-group
# req = json.loads(REQUEST)
msg = ec2_secg_check()
resp = send_email('hydranoob000@gmail.com', 'Hydra', 'Vulnerability Manager: Critical SG Rule found', msg, 'hydranoob001@gmail.com')
print(resp)
msg

{'result': 'OK!, Email has been sent to user successfully', 'execution_status': 'SUCCESS'}


'Found malicious rule for port: 3389<br>Rule Name: hydra<br>Link to rule: https://ap-south-1.console.aws.amazon.com/ec2/home?region=ap-south-1#SecurityGroups:search=hydra'